# 1.

In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=45e6912cd7c0dbe2ff3e4ef1f561b73cb3cb0bc9963eddd0b7f9c99dd3da8bcb
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


## 1.1)-

In [2]:
from pyspark import SparkConf, SparkContext, SQLContext
from pyspark.sql.functions import *
from pyspark.sql.types import StringType,IntegerType,StructField,StructType

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
config = SparkConf() \
    .setAppName('exam')

sc = SparkContext(conf=config)


In [9]:
rdd = sc.textFile("/content/drive/My Drive/Exam2_Data.txt")

## 1.2)-

In [11]:
#Display the result
hdr = rdd.first()
filtered = rdd.filter(lambda row: row != hdr)
print("\n\n --- Le nombre total d'enregistrements dans le RDD est: = ",filtered.count()," --- \n\n")



 --- Le nombre total d'enregistrements dans le RDD est: =  100  --- 




## 1.3)-

In [13]:
#Display the result
cols = hdr.split("\t")
idx = cols.index("payment_method")
cols

['date', 'time', 'store', 'product_type', 'cost', 'payment_method']

In [24]:
visa = filtered.map(lambda row : row.split("\t")).map(lambda row : row[idx]).filter(lambda row : row == 'Visa').map(lambda row : str(row))
print("\n\n --- Le nombre total d'enregistrements dans le paiement est avec une carte Visa est: = ",visa.count()," --- \n\n")



 --- Le nombre total d'enregistrements dans le paiement est avec une carte Visa est: =  27  --- 




## 1.4)-

In [30]:
#Display the result
cost_idx = cols.index("cost")
pairs = filtered.map(lambda row : row.split("\t")).map(lambda row: (row[cost_idx], row[idx])).filter(lambda row : row[0].replace(".","").isnumeric()).filter(lambda row : row[1] == 'Visa').map(lambda row : (float(row[0]), row[1]))


(153.57, 'Visa')

In [31]:
mean_cost = pairs.map(lambda row: row[0]).mean()
print("\n\n --- Le coût moyen d'un paiement par carte Visa est: = ",mean_cost," --- \n\n")




 --- Le coût moyen d'un paiement par carte Visa est: =  240.6714814814815  --- 




# 2.

In [34]:
sqlContext = SQLContext(sc)

## 2.1)-

In [36]:
#Display the result
df = sqlContext.read.csv('/content/drive/My Drive/Exam2_Data.txt', header=True, inferSchema=True, sep='\t')
df

DataFrame[date: date, time: timestamp, store: string, product_type: string, cost: double, payment_method: string]

In [37]:
df.printSchema()

root
 |-- date: date (nullable = true)
 |-- time: timestamp (nullable = true)
 |-- store: string (nullable = true)
 |-- product_type: string (nullable = true)
 |-- cost: double (nullable = true)
 |-- payment_method: string (nullable = true)



## 2.2)-

In [39]:
df.registerTempTable("payments")

## 2.3)-

2.3.1)- (1)

In [40]:
#Display the result
sqlContext.sql("""
  SELECT * from payments
  ORDER BY cost DESC
""").show(10)

+----------+-------------------+-----------+--------------------+------+--------------+
|      date|               time|      store|        product_type|  cost|payment_method|
+----------+-------------------+-----------+--------------------+------+--------------+
|2012-01-01|2024-05-30 09:00:00| Pittsburgh|        Pet Supplies|493.51|      Discover|
|2012-01-01|2024-05-30 09:00:00|    Buffalo|    Women's Clothing|483.82|          Visa|
|2012-01-01|2024-05-30 09:04:00|    Norfolk|Consumer Electronics|480.24|          Cash|
|2012-01-01|2024-05-30 09:01:00| Pittsburgh|      Sporting Goods|475.26|          Amex|
|2012-01-01|2024-05-30 09:02:00|  Riverside|                 CDs|472.71|          Cash|
|2012-01-01|2024-05-30 09:00:00|     Austin|             Cameras|469.63|    MasterCard|
|2012-01-01|2024-05-30 09:01:00|     Fresno|                 CDs|466.64|    MasterCard|
|2012-01-01|2024-05-30 09:03:00|    Wichita|      Men's Clothing|460.38|    MasterCard|
|2012-01-01|2024-05-30 09:04:00|

2.3.1)- (2)


In [43]:
#Display the result
df.sort(desc("cost")).show(10)

+----------+-------------------+-----------+--------------------+------+--------------+
|      date|               time|      store|        product_type|  cost|payment_method|
+----------+-------------------+-----------+--------------------+------+--------------+
|2012-01-01|2024-05-30 09:00:00| Pittsburgh|        Pet Supplies|493.51|      Discover|
|2012-01-01|2024-05-30 09:00:00|    Buffalo|    Women's Clothing|483.82|          Visa|
|2012-01-01|2024-05-30 09:04:00|    Norfolk|Consumer Electronics|480.24|          Cash|
|2012-01-01|2024-05-30 09:01:00| Pittsburgh|      Sporting Goods|475.26|          Amex|
|2012-01-01|2024-05-30 09:02:00|  Riverside|                 CDs|472.71|          Cash|
|2012-01-01|2024-05-30 09:00:00|     Austin|             Cameras|469.63|    MasterCard|
|2012-01-01|2024-05-30 09:01:00|     Fresno|                 CDs|466.64|    MasterCard|
|2012-01-01|2024-05-30 09:03:00|    Wichita|      Men's Clothing|460.38|    MasterCard|
|2012-01-01|2024-05-30 09:04:00|

2.3.2)- (1)

In [46]:
#Display the result
sqlContext.sql("""
  SELECT product_type, SUM(cost) as cost_count
  FROM payments
  GROUP BY product_type
""").show()

+--------------------+------------------+
|        product_type|        cost_count|
+--------------------+------------------+
| Children's Clothing|            999.49|
|      Sporting Goods|            1641.7|
|                 CDs|           1326.29|
|           Computers|            1248.9|
|Consumer Electronics|           1471.77|
|   Health and Beauty| 868.1500000000001|
|        Pet Supplies|           1266.69|
|                DVDs|           2020.68|
|                Baby|           1182.13|
|              Crafts|488.44000000000005|
|    Women's Clothing|1501.8899999999999|
|         Video Games|            846.48|
|               Books|1568.4899999999998|
|               Music| 672.5799999999999|
|      Men's Clothing|           2268.33|
|             Cameras|           1884.91|
|              Garden|            779.48|
|                Toys|1169.0800000000002|
+--------------------+------------------+



2.3.2)- (2)

In [47]:
#Display the result
df.groupBy(df.product_type).sum().show()

+--------------------+------------------+
|        product_type|         sum(cost)|
+--------------------+------------------+
| Children's Clothing|            999.49|
|      Sporting Goods|            1641.7|
|                 CDs|           1326.29|
|           Computers|            1248.9|
|Consumer Electronics|           1471.77|
|   Health and Beauty| 868.1500000000001|
|        Pet Supplies|           1266.69|
|                DVDs|           2020.68|
|                Baby|           1182.13|
|              Crafts|488.44000000000005|
|    Women's Clothing|1501.8899999999999|
|         Video Games|            846.48|
|               Books|1568.4899999999998|
|               Music| 672.5799999999999|
|      Men's Clothing|           2268.33|
|             Cameras|           1884.91|
|              Garden|            779.48|
|                Toys|1169.0800000000002|
+--------------------+------------------+



# 3.

In [83]:
df = df.na.fill(0)

## 3.1)-


In [84]:
train_df, test_df = df.randomSplit([0.8, 0.2], seed=123)

## 3.2)-

In [99]:
from pyspark.ml.feature import StringIndexer

indexer = StringIndexer(inputCol="store", outputCol="store_idx", handleInvalid="keep")
indexed_df = indexer.fit(train_df).transform(train_df)

indexer2 = StringIndexer(inputCol="product_type", outputCol="product_type_idx", handleInvalid="keep")
indexed_df = indexer2.fit(indexed_df).transform(indexed_df)

indexer3 = StringIndexer(inputCol="payment_method", outputCol="payment_method_idx", handleInvalid="keep")
indexed_df = indexer3.fit(indexed_df).transform(indexed_df)

indexed_df

DataFrame[date: date, time: timestamp, store: string, product_type: string, cost: double, payment_method: string, store_idx: double, product_type_idx: double, payment_method_idx: double]

In [100]:
from pyspark.ml.feature import OneHotEncoder
encoder = OneHotEncoder(inputCol="store_idx", outputCol="store_encoded")
encoded_df = encoder.fit(indexed_df).transform(indexed_df)

encoder2 = OneHotEncoder(inputCol="product_type_idx", outputCol="product_type_encoded")
encoded_df = encoder2.fit(encoded_df).transform(encoded_df)

encoder3 = OneHotEncoder(inputCol="payment_method_idx", outputCol="payment_method_encoded")
encoded_df = encoder3.fit(encoded_df).transform(encoded_df)

encoded_df

DataFrame[date: date, time: timestamp, store: string, product_type: string, cost: double, payment_method: string, store_idx: double, product_type_idx: double, payment_method_idx: double, store_encoded: vector, product_type_encoded: vector, payment_method_encoded: vector]

## 3.3)-

In [113]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(
    inputCols=['store_encoded', 'product_type_encoded', 'payment_method_encoded', 'cost'],
    outputCol="features")

transformed_df = assembler.transform(encoded_df)

## 3.4)-

In [114]:
from pyspark.ml import Pipeline

stages = [indexer, indexer2, indexer3, encoder, encoder2, encoder3, assembler]

pipeline = Pipeline(stages=stages)

## 3.5)-

In [115]:
#Display the result
pipeline_model = pipeline.fit(train_df)

transformed_df = pipeline_model.transform(train_df)


In [116]:
transformed_df

DataFrame[date: date, time: timestamp, store: string, product_type: string, cost: double, payment_method: string, store_idx: double, product_type_idx: double, payment_method_idx: double, store_encoded: vector, product_type_encoded: vector, payment_method_encoded: vector, features: vector]

## 3.6)-

In [117]:
from pyspark.ml.clustering import KMeans

kmeans = KMeans(featuresCol='features', k=5)

## 3.7)-

In [118]:
model = kmeans.fit(transformed_df)

## 3.8)-

In [119]:
#Display the result
pipeline_test_model = pipeline.fit(test_df)

transformed_test_df = pipeline_model.transform(test_df)

In [120]:
predictions = model.transform(transformed_test_df)

In [121]:
predictions.show(5)

+----------+-------------------+----------+--------------------+------+--------------+---------+----------------+------------------+---------------+--------------------+----------------------+--------------------+----------+
|      date|               time|     store|        product_type|  cost|payment_method|store_idx|product_type_idx|payment_method_idx|  store_encoded|product_type_encoded|payment_method_encoded|            features|prediction|
+----------+-------------------+----------+--------------------+------+--------------+---------+----------------+------------------+---------------+--------------------+----------------------+--------------------+----------+
|2012-01-01|2024-05-30 09:00:00|    Boston|             Cameras|418.94|          Amex|     56.0|             8.0|               4.0|     (56,[],[])|      (18,[8],[1.0])|         (5,[4],[1.0])|(80,[64,78,79],[1...|         4|
|2012-01-01|2024-05-30 09:00:00|Fort Worth|    Women's Clothing|153.57|          Visa|     24.0|    

## 3.9)-

In [122]:
#Display the result
from pyspark.ml.evaluation import ClusteringEvaluator

evaluator = ClusteringEvaluator()


evaluator = evaluator.setMetricName('silhouette').setFeaturesCol('features').setPredictionCol('prediction')

silhouette = evaluator.evaluate(predictions)

print("Silhouette coefficient: ", silhouette)

Silhouette coefficient:  0.5710933726049439
